# PyBADS Example 1: Basic usage

In this introductory example, we will show a simple usage of Bayesian Adaptive Direct Search (BADS) to perform optimization of a synthetic target function.

This notebook is Part 1 of a series of notebooks in which we present various example usages for BADS with the PyBADS package.

In [1]:
import numpy as np
from pybads.bads import BADS

## 0. What is (Py)BADS?

BADS is a fast hybrid Bayesian optimization algorithm designed to solve difficult optimization problems, in particular related to parameter estimation -- aka model fitting -- of computational models (e.g., via maximum-likelihood or maximum-a-posteriori estimation). **PyBADS is its Python implementation**.

BADS has been intensively tested for fitting a variety of computational models, and is currently used by many research groups around the world (see [Google Scholar](https://scholar.google.co.uk/scholar?cites=7209174494000095753&as_sdt=2005&sciodt=0,5&hl=en) for many example applications). In our benchmark with real model-fitting problems, BADS performed on par or better than many other common and state-of-the-art optimizers, as shown in the [original BADS paper](https://arxiv.org/abs/1705.04405).

BADS is particularly recommended when:
- the objective function landscape is rough (nonsmooth), typically due to numerical approximations or noise;
- the objective function is moderately expensive to compute (e.g., more than 0.1 second per function evaluation);
- the gradient is unavailable;
- the number of input parameters is up to about `D = 20` or so.

BADS requires no specific tuning and runs off-the-shelf like other built-in optimizers (e.g., from `scipy.optimize.minimize`).

*Note*: If you are interested in estimating posterior distributions (i.e., uncertainty and error bars) over model parameters, and not just point estimates, you might also want to check out Variational Bayesian Monte Carlo for Python (PyVBMC), a package for Bayesian posterior and model inference which can be used in synergy with PyBADS.

### Optimization problem

Formally, the goal of BADS is to *minimize* a target (or objective) function $f(\mathbf{x}): \mathbb{R}^D \rightarrow \mathbb{R}$, for $\mathbf{x} \in \mathbb{R}^D$,
$$
\mathbf{x}^\star = \arg\min_\mathbf{x} f(\mathbf{x}) \qquad \text{with} \; \text{lb}_d \le x_d \le \text{ub}_d \; \text{ for } 1\le d \le D,
$$
where $D$ is the dimensionality of the problem and `lb`, `ub` are arrays representing lower/upper bound constraints, which can be set to infinite for unbounded parameters.

## 1. Problem setup

Here we show PyBADS at work on [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) in 2D as target function.  

We specify wide hard bounds and tighter plausible bounds that (hopefully) contain the solution. 
- Hard lower/upper bounds `lb`, `ub` are the actual optimization bounds; PyBADS will not evaluate the target function outside these bounds, but might evaluate the target on the bounds. You can use `-np.inf` and `np.inf` for unbounded parameters.
- Plausible lower/upper bounds `plb`, `pub` represent our best guess at bounding the region where the solution might lie. The plausible bounds do not change the optimization problem, but help define the initial exploration and hyperparameters of PyBADS.

We set as starting point for the optimization $\mathbf{x}_0 = (0, 0)$.

In [2]:
def rosenbrocks_fcn(x):
    """Rosenbrock's 'banana' function in any dimension."""
    x_2d = np.atleast_2d(x)
    return np.sum(100 * (x_2d[:, 0:-1]**2 - x_2d[:, 1:])**2 + (x_2d[:, 0:-1]-1)**2, axis=1)

target = rosenbrocks_fcn;

lb = np.array([-20, -20])     # Lower bounds
ub = np.array([20, 20])       # Upper bounds
plb = np.array([-5, -5])      # Plausible lower bounds
pub = np.array([5, 5])        # Plausible upper bounds
x0 = np.array([0, 0]);        # Starting point

## 2. Initialize and run the optimization

Then, we initialize a `bads` instance which takes care of the optimization. For now, we use default options.  
To run the optimization, we simply call `bads.optimize()`, which returns an `OptimizeResult` object.

In [3]:
bads = BADS(target, x0, lb, ub, plb, pub)
optimize_result = bads.optimize()

Beginning optimization of a DETERMINISTIC objective function

 Iteration    f-count         f(x)           MeshScale          Method             Actions
     0           2               1               1                                 Uncertainty test
     0           6               1               1         Initial mesh            Initial points
     0          10               1             0.5         Refine grid             Train
     1          18               1            0.25         Refine grid             Train
     2          19       0.0600236            0.25     Successful search (ES-wcm)        
     2          24      0.00320034            0.25     Incremental search (ES-ell)        
     2          30      0.00320034           0.125         Refine grid             Train
     3          34     0.000634688           0.125     Incremental search (ES-wcm)        
     3          38     0.000634688          0.0625         Refine grid             
     4          46     0.0

## 3. Results and conclusions

We examine now the result of the optimization stored in `optimize_result`. The most important keys are the minimum, `optimize_result['x']`, and the value of the function at the solution, `optimize_result['fval']`.

In [4]:
x_min = optimize_result['x']
fval = optimize_result['fval']

print(f"BADS minimum at: x_min = {x_min.flatten()}, fval = {fval:.4g}")
print(f"total f-count: {optimize_result['func_count']}, time: {round(optimize_result['total_time'], 2)} s")

BADS minimum at: x_min = [1.01732125 1.0353968 ], fval = 0.0003207
total f-count: 71, time: 4.07 s


The true minimum of the Rosenbrock function is at $\textbf{x}^\star = [1, 1]$, where $f^\star = 0$.  

In conclusion, PyBADS found the solution with a fairly small number of function evaluations (`f-count`), which is particularly important if the target function is mildly-to-very expensive to compute as in many computational models.

*Note*: PyBADS by default does not aim for extreme numerical precision of the target (e.g., beyond the 2nd or 3rd decimal place), since in most realistic model-fitting problems a higher resolution is typically pointless, e.g. due to noise or variability in the data.

## Example 1: Full code

See [here](./scripts/pybads_example_1_basic_usage.py) for a Python file with the code used in this example, with no extra fluff.